## Prepare


In [2]:
from azureml.core import Workspace, Dataset
from sklearn.model_selection import train_test_split


# connect to your workspace
ws = Workspace.from_config()

# Get a named dataset from the current workspace
dataset = Dataset.get_by_name(ws, name="sentiment140")
df = dataset.to_pandas_dataframe()
df.columns = ["target", "text"]

# Replace target values with labels
df.target.replace(
    {
        0: "NEGATIVE",
        2: "NEUTRAL",
        4: "POSITIVE",
    },
    inplace=True,
)
df.target.replace(
    {
        "NEGATIVE": 0,
        "POSITIVE": 1,
    },
    inplace=True,
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df.text,
    df.target,
    test_size=0.2,
    stratify=df.target,
    random_state=42,
)

## Train


In [3]:
model_name = "Bidirectional-LSTM"

In [ ]:
# create the model
model_name = "Bidirectional-LSTM"
experiment_name = "oc-p7-notebook"


import mlflow


# set up MLflow to track the metrics
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)
mlflow.autolog()


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,
    TextVectorization,
    Dense,
    Dropout,
    Embedding,
    Bidirectional,
    LSTM,
)
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.metrics import AUC


# Model constants.
max_features = 10000
sequence_length = 30
embedding_dim = 100
rnn_units = 100


# Define vectorizer
vectorize_layer = TextVectorization(
    output_mode="int",
    max_tokens=max_features,
    output_sequence_length=sequence_length,
)
vectorize_layer.adapt(
    df.text,
    batch_size=128,
)

# define NN model
model = Sequential(name=model_name)
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)

# Embedding layer
model.add(
    Embedding(
        max_features,
        embedding_dim,
        input_length=sequence_length,
    )
)

# Bidirectional LSTM layer
model.add(Bidirectional(LSTM(units=rnn_units, dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(units=rnn_units, dropout=0.2)))

# Dense layers
model.add(Dense(100, input_shape=(max_features,), activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation="relu"))

# Classification layer
model.add(Dense(1, activation="sigmoid"))

# compile NN network
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        AUC(curve="ROC", name="ROC_AUC"),
        AUC(curve="PR", name="AP"),
    ],
)

# train the model
with mlflow.start_run() as run:
    # fit NN model
    model.fit(
        X_train,
        y_train,
        epochs=10,
        batch_size=128,
        validation_split=0.2,
        callbacks=[
            TensorBoard(log_dir=f"logs/{model.name}"),
            EarlyStopping(monitor="val_loss", patience=2),
        ],
        workers=4,
        use_multiprocessing=True,
    )

    # register the model
    model_uri = "runs:/{}/model".format(run.info.run_id)
    model = mlflow.register_model(model_uri, model_name)

## Deploy


In [3]:
import uuid
from azureml.core import Workspace, Environment, Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig


version = "oc-p7-" + str(uuid.uuid4())[:8]

env = Environment.from_conda_specification(
    name=version, file_path="conda_dependencies.yml"
)

model = Model(ws, model_name)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
)

inference_config = InferenceConfig(entry_script="score.py", environment=env)

service = Model.deploy(
    workspace=ws,
    name=version,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-26 08:19:52+00:00 Creating Container Registry if not exists.
2022-01-26 08:19:52+00:00 Registering the environment.
2022-01-26 08:19:55+00:00 Use the existing image.
2022-01-26 08:19:55+00:00 Generating deployment configuration.
2022-01-26 08:19:57+00:00 Submitting deployment to compute.
2022-01-26 08:20:03+00:00 Checking the status of deployment oc-p7-f641b2f1..
2022-01-26 08:23:05+00:00 Checking the status of inference endpoint oc-p7-f641b2f1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Test


In [26]:
# send raw HTTP request to test the web service.
import numpy as np
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test) - 1)
text = X_test.iloc[random_index]
y_true = y_test.iloc[random_index]

headers = {"Content-Type": "application/json"}
input_data = '{"data": ["' + X_test.iloc[random_index] + '"]}'
resp = requests.post(service.scoring_uri, input_data, headers=headers)

y_pred = float(resp.text[2:-2])


# Display the text
print(f'Text : "{text}"')

if round(y_pred) == y_true:
    print(
        f"✅ Predicted review sentiment correct : {round(y_pred,3)} (pred) vs. {y_true} (true)"
    )
else:
    print(
        f"❌ Predicted review sentiment incorrect : {round(y_pred,3)} (pred) vs. {y_true} (true)"
    )

Text : "Eating some delicious cake "
✅ Predicted review sentiment correct : 0.958 (pred) vs. 1 (true)
